<a href="https://colab.research.google.com/github/nitinlodhi019/Deep-Learning/blob/main/Hyperparameter_Tuning/Hyperparameter_Tuning_a_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Here i can change my code on on the basis of my need and add more parameters fro tuning purpose**

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_csv("/content/diabetes.csv")
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
data.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [ ]:
X = data.iloc[:,:-1].values
y = data.iloc[:, -1].values

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)
X

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout

In [ ]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=8))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 50ms/step - accuracy: 0.5966 - loss: 0.7235 - val_accuracy: 0.6429 - val_loss: 0.6972
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6430 - loss: 0.6698 - val_accuracy: 0.6558 - val_loss: 0.6513
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.6516 - loss: 0.6384 - val_accuracy: 0.6753 - val_loss: 0.6123
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6736 - loss: 0.6064 - val_accuracy: 0.7273 - val_loss: 0.5823
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7065 - loss: 0.5675 - val_accuracy: 0.7532 - val_loss: 0.5616
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7202 - loss: 0.5486 - val_accuracy: 0.7857 - val_loss: 0.5420
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7114 - loss: 0.5500 - val_accuracy: 0.7987 - val_loss: 0.5270
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.7388 - loss: 0.5197 - val_accuracy: 0.8117 - va

In [ ]:
##pip install keras-tuner

## Find Best optimizer

In [ ]:
import keras_tuner as kt

In [ ]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(32, activation='relu', input_dim=8))
  model.add(Dense(1, activation='sigmoid'))

  optimizer = hp.Choice('optimizer',values = ['adam', 'sgd', 'rmsprop', 'adadelta'])
  model.compile(optimizer = optimizer, loss='binary_crossentropy', metrics = ['accuracy'])
  return model

In [ ]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5)  ##it's a class

Reloading Tuner from ./untitled_project/tuner0.json


In [ ]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

In [ ]:
tuner.get_best_hyperparameters()[0].values  ## it will be return best HP

{'optimizer': 'adam'}

In [ ]:
model = tuner.get_best_models(num_models=1)[0]
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
## initial_epoch= 6, we are taking because our model run already 5 epochs so i want to start from 6
model.fit(X_train, y_train, epochs=100, batch_size=32, initial_epoch= 6, validation_data=(X_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.7106 - loss: 0.5663 - val_accuracy: 0.7857 - val_loss: 0.5151
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7217 - loss: 0.5426 - val_accuracy: 0.7987 - val_loss: 0.4968
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7442 - loss: 0.5072 - val_accuracy: 0.7922 - val_loss: 0.4845
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7150 - loss: 0.4964 - val_accuracy: 0.7857 - val_loss: 0.4777
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7289 - loss: 0.5123 - val_accuracy: 0.7857 - val_loss: 0.4709
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7375 - loss: 0.4979 - val_accuracy: 0.7857 - val_loss: 0.4667
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7564 - loss: 0.4648 - val_accuracy: 0.7792 - val_loss: 0.4632
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7653 - loss: 0.4744 - val_accuracy: 0.77

## Find best Neurons

In [ ]:
def build_model1(hp):
    model = Sequential()
    units = hp.Int('units', min_value=8, max_value=128, step=8)
    model.add(Dense(units=units, activation='relu', input_dim=8))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
tuner1 = kt.RandomSearch(
    build_model1,
    objective='val_accuracy',
    max_trials=5,
    directory='my_dir',
    project_name='intro'
)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
tuner1.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))
## i can find details in directory section with each trials

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.7857142686843872

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 00m 17s


In [ ]:
best_hps = tuner1.get_best_hyperparameters(num_trials=1)[0]
print("Best units (neurons):", best_hps.get('units'))

Best units (neurons): 56


In [ ]:
model1 = tuner1.get_best_models(num_models=1)[0]
model1.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 56)                  │             504 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              57 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 561 (2.19 KB)

 Trainable params: 561 (2.19 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model1.fit(X_train, y_train, epochs=100, initial_epoch= 6, validation_data=(X_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.7155 - loss: 0.5688 - val_accuracy: 0.7662 - val_loss: 0.5086
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7916 - loss: 0.5079 - val_accuracy: 0.7792 - val_loss: 0.4863
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7817 - loss: 0.4928 - val_accuracy: 0.7857 - val_loss: 0.4736
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7915 - loss: 0.4821 - val_accuracy: 0.7987 - val_loss: 0.4633
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7812 - loss: 0.4713 - val_accuracy: 0.7987 - val_loss: 0.4573
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7680 - loss: 0.4883 - val_accuracy: 0.7987 - val_loss: 0.4538
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7678 - loss: 0.4627 - val_accuracy: 0.8052 - val_loss: 0.4505
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7944 - loss: 0.4326 - val_accuracy: 0.78

## Find Numbers of layers

In [ ]:
def build_model2(hp):
  model = Sequential()
  model.add(Dense(72, activation='relu', input_dim=8))

  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
    model.add(Dense(72, activation='relu'))

  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [ ]:
tuner2 = kt.RandomSearch(build_model2,
    objective='val_accuracy',
    max_trials=3,
    directory='my_dir',
    project_name='num_layers')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
tuner2.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 3 Complete [00h 00m 05s]
val_accuracy: 0.8116883039474487

Best val_accuracy So Far: 0.8311688303947449
Total elapsed time: 00h 00m 19s


In [ ]:
tuner2.get_best_hyperparameters()[0].values

{'num_layers': 8}

In [ ]:
model2 = tuner2.get_best_models(num_models=1)[0]
model2.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 42 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 72)                  │             648 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 72)                  │           5,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 72)                  │           5,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 72)                  │           5,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 72)                  │           5,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 72)                  │           5,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 72)                  │           5,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 72)                  │           5,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 72)                  │           5,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 1)                   │              73 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 42,769 (167.07 KB)

 Trainable params: 42,769 (167.07 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model2.fit(X_train, y_train, epochs=100, initial_epoch= 6, validation_data=(X_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step - accuracy: 0.8072 - loss: 0.4136 - val_accuracy: 0.7922 - val_loss: 0.4806
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8048 - loss: 0.4173 - val_accuracy: 0.7987 - val_loss: 0.5007
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7939 - loss: 0.4019 - val_accuracy: 0.7792 - val_loss: 0.5016
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8435 - loss: 0.3594 - val_accuracy: 0.7857 - val_loss: 0.5330
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8517 - loss: 0.3771 - val_accuracy: 0.8117 - val_loss: 0.5110
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8581 - loss: 0.3405 - val_accuracy: 0.7403 - val_loss: 0.6126
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8636 - loss: 0.3186 - val_accuracy: 0.7792 - val_loss: 0.5692
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9016 - loss: 0.2837 - val_accuracy: 0.7

## Find multiple Parameter with one code

In [ ]:
def build_model3(hp):
  model = Sequential()
  counter = 0

  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
    if counter == 0:
      model.add(
          Dense(
              hp.Int('units' + str(i), min_value=8, max_value=128, step=8),     ##we need a new string each time
              activation=hp.Choice('activation'+str(i), values=['relu', 'tanh', 'sigmoid']),
              input_dim=8
              )
          )
      model.add(Dropout(hp.Choice('dropout' + str(i), values=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])))
    else:
     model.add(
          Dense(
              hp.Int('units' + str(i), min_value=8, max_value=128, step=8),     ##we need a new string each time
              activation=hp.Choice('activation'+str(i), values=['relu', 'tanh', 'sigmoid'])
              )
          )
     model.add(Dropout(hp.Choice('dropout' + str(i), values=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])))
     counter +=1

  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer=hp.Choice('optimizer',values = ['adam', 'sgd', 'rmsprop', 'adadelta']),
                loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [ ]:
tuner3 = kt.RandomSearch(build_model3, objective='val_accuracy', max_trials=5, directory='my_dir', project_name='Final')

Reloading Tuner from my_dir/Final/tuner0.json


In [ ]:
tuner3.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

In [ ]:
tuner3.get_best_hyperparameters()[0].values

{'num_layers': 6,
 'units0': 64,
 'activation0': 'tanh',
 'optimizer': 'adam',
 'units1': 8,
 'activation1': 'relu',
 'units2': 8,
 'activation2': 'relu',
 'units3': 8,
 'activation3': 'relu',
 'units4': 8,
 'activation4': 'relu',
 'units5': 8,
 'activation5': 'relu'}

In [ ]:
model3 = tuner3.get_best_models(num_models=1)[0]
model3.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 30 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │             576 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 8)                   │             520 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 8)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 8)                   │              72 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 8)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 8)                   │              72 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 8)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 8)                   │              72 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 8)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 8)                   │              72 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 8)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 1)                   │               9 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,393 (5.44 KB)

 Trainable params: 1,393 (5.44 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(X_train, y_train, epochs=100, initial_epoch= 5, validation_data=(X_test, y_test))

Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8332 - loss: 0.3702 - val_accuracy: 0.8052 - val_loss: 0.4744
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8440 - loss: 0.3441 - val_accuracy: 0.8052 - val_loss: 0.4749
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8473 - loss: 0.3500 - val_accuracy: 0.8052 - val_loss: 0.4761
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8355 - loss: 0.3454 - val_accuracy: 0.8117 - val_loss: 0.4774
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8624 - loss: 0.3135 - val_accuracy: 0.8052 - val_loss: 0.4773
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8465 - loss: 0.3395 - val_accuracy: 0.8052 - val_loss: 0.4765
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8518 - loss: 0.3303 - val_accuracy: 0.8052 - val_loss: 0.4731
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8666 - loss: 0.3242 - val_accuracy: 0.805